## Imports

In [1]:
import os
import json

parent_path = os.path.abspath("./../")
if os.path.basename(parent_path) == 'vdp':
    os.chdir(parent_path)

from glob import glob
import vdp
import re
import numpy as np


from vdp.vocabulary import FOSort, FOFunction
from vdp.fomodel import FOElement, FOModel
from vdp.exceptions import PartialInterpretationException




## Generate config.json files

In [36]:
# define the scene graph and the fo model generation configs.
sgconfig = {
"output_dir": "./../data/sg_processed",
"glove_path": "./../../checkpoints/glove",
"model_path": "./../../checkpoints/model",
"sg_tools_rel_path": "./tools/relation_test_net.py",
"sg_config_path": "./configs/e2e_relation_X_101_32_8_FPN_1x.yaml",
"cuda_device_port": 0,
'n_proc': 1
}

foconfig = {
    "input_dir": "./data/sg_processed",
    "output_dir": "./data/sg_ir",
    "rel_topk": 20, 
    "box_topk": 20
}

In [18]:
# write config files to memory.
vdp_puzzles =  glob("./data/raw/*")
for folder in vdp_puzzles:
    train = glob(folder + "/train*")
    test  = glob(folder + "/test*")
    name = os.path.basename(folder)
    config = {
        'name' : name,
        'train' : list(train),
        'test' : list(test),
        'sg_config' : sgconfig,
        'fo_config' : foconfig
    }
    json.dump(config, open(f"./config/{name}.json", 'w'))
    

NameError: name 'sgconfig' is not defined

## YOLO $\to$ IR

In [2]:
def yolo_to_ir(yolo_folders, output_folder):
    for folder_path in yolo_folders:
        name = os.path.basename(folder_path)
        output_dir =f"{output_folder}/{name}" 
        os.makedirs(output_dir, exist_ok=True)
        fo_models = vdp.utils.get_yolo_fo_models(folder_path)
        
        for batch in ['test', 'train']:
            batch_path = f"{output_dir}/{batch}"
            os.makedirs(batch_path, exist_ok=True)
            for img_path, fo_model in fo_models:
                if os.path.basename(img_path).split("_")[0] == batch:
                    json_name = os.path.basename(img_path).replace('.jpg', '_model.json')
                    output_json = f"{batch_path}/{json_name}"
                    with open(output_json, 'w') as fp:
                        json.dump(fo_model, fp)


yolo_to_ir(glob("./vdp_smt/*/*"), './data/yolo_ir')


## SGG $\to$ IR

In [16]:
def sgg_to_ir(input_dirs, output_dir, box_topk, rel_topk, raw_img_dir):
    
    for input_dir in input_dirs:
        name = os.path.basename(input_dir)
        for batch_dir in ['train', 'test']:
            sg_input_dir = input_dir + f"/{batch_dir}"
            fo_output_dir = output_dir + f"/{name}/{batch_dir}"
            os.makedirs(fo_output_dir, exist_ok=True)
            fo_models = vdp.utils.get_sgg_fo_models(sg_input_dir, box_topk=box_topk, rel_topk=rel_topk, raw_img_dir=raw_img_dir)
            
            for img_path, fo_model in fo_models:
                output_json = f"{fo_output_dir}/{os.path.basename(img_path).replace('.jpg', '_model.json')}"
                with open(output_json, 'w') as fp:
                    json.dump(fo_model, fp)

        

sgg_to_ir(glob("./data/sg_processed/*"), "./data/sg_ir", 20, 20, "./data/raw")

{'sorts': ['object', 'label', 'scores'],
 'predicates': {'left_above': '(object, object)',
  'right_below': '(object, object)',
  'has label': '(object, label)',
  'has score': '(object, object, score)'},
 'elements': {'object': ['obj_cat', 'obj_sofa'],
  'label': ['cat', 'sofa'],
  'scores': [0.6272, 0.6272]},
 'interpretation': {'left_above': [('obj_sofa', 'obj_cat')],
  'right_below': [('obj_cat', 'obj_sofa')],
  'has label': [('obj_sofa', 'sofa'), ('obj_cat', 'cat')],
  'has score': [('obj_sofa', 'obj_cat', 0.6272),
   ('obj_cat', 'obj_sofa', 0.6272)]},
 'raw': [['obj', 'sofa', 'obj', 'cat', 'left_above', 0.6272],
  ['obj', 'cat', 'obj', 'sofa', 'right_below', 0.6272]]}

In [7]:

relations = {}
for label1, pred1, bb1 in img_params:
    for label2, pred2, bb2 in img_params:
        if label1 != label2:
            c1 = centroid(bb1)
            c2 = centroid(bb2)
            relations[(label1, label2)] = relate(c1, c2)
        else:
            relations[(label1, label2)] = ['labelOf']
            
labels, preds, bbs = zip(*img_params)

relations


{('bed', 'bed'): ['labelOf'],
 ('bed', 'sofa'): ['right', 'above'],
 ('bed', 'cat'): ['left', 'above'],
 ('sofa', 'bed'): ['left', 'below'],
 ('sofa', 'sofa'): ['labelOf'],
 ('sofa', 'cat'): ['left', 'above'],
 ('cat', 'bed'): ['right', 'below'],
 ('cat', 'sofa'): ['right', 'below'],
 ('cat', 'cat'): ['labelOf']}

In [8]:
vdpobject = FOSort('Object')
vdplabel = FOSort('Label')
vdpbool = FOSort('Bool')


In [9]:
vdpconstants = dict()
for label in labels:
    if label not in vdpconstants:
        vdpconstants[label] = FOFunction(label, tuple([vdplabel]))
    
    
unique_labels = list(vdpconstants.keys()) # @TODO REPLACE!!!!


In [10]:
# on = FOFunction('On', (vdpobject, vdpobject, vdpbool))
relation_labels = ['left', 'right', 'above', 'below']
vdprelations = dict()

for rl in relation_labels:
    vdprelations[rl] = FOFunction(rl, (vdpobject, vdpobject, vdpbool))


vdprelations['labelOf'] = FOFunction('labelOf', (vdpobject, vdplabel, vdpbool))
vdprelations

{'left': <vdp.vocabulary.FOFunction at 0x10889af10>,
 'right': <vdp.vocabulary.FOFunction at 0x10889a250>,
 'above': <vdp.vocabulary.FOFunction at 0x10889ab90>,
 'below': <vdp.vocabulary.FOFunction at 0x10889a790>,
 'labelOf': <vdp.vocabulary.FOFunction at 0x10889ad10>}

In [11]:
vdpvocabulary = (set([vdpobject, vdplabel]), set(list(vdpconstants.values()) + list(vdprelations.values())))


In [12]:
vdpobjects = {label : FOElement(label + "_object", vdpobject) for label in labels}
vdplabels = {label : FOElement(label + "_label", vdplabel) for label in unique_labels}

vdpelements = {'Object': set(vdpobjects.values()), 'Label': set(vdplabels.values())}

In [13]:
labelconst_interpretation = {const : vdplabels[label] for label, const in zip(unique_labels, list(vdpconstants.values()))}
labelconst_interpretation

{<vdp.vocabulary.FOFunction at 0x10889a650>: <vdp.fomodel.FOElement at 0x111a026d0>,
 <vdp.vocabulary.FOFunction at 0x10889a2d0>: <vdp.fomodel.FOElement at 0x111a02810>,
 <vdp.vocabulary.FOFunction at 0x10889a510>: <vdp.fomodel.FOElement at 0x111a02910>}

In [14]:

    

rel_interpretation = dict()
# for rel_sym, rel in vdprelations.items():
#     if 'labelOf' in rel_sym:
#         rel_interpretation[rel] = {(obj, lab) : rel_sym in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}
#     else:
#         rel_interpretation[rel] = {(obj1, obj2) : rel_sym in relations[(obj1_sym, obj2_sym)]  for obj1_sym, obj1 in vdpobjects.items() for obj2_sym, obj2 in vdpobjects.items() if obj1_sym != obj2_sym}

        
rel_interpretation[vdprelations['labelOf']] = {(obj, lab) : 'labelOf' in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}


rel_interpretation

{<vdp.vocabulary.FOFunction at 0x10889ad10>: {(<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a026d0>): True,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02810>): True,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02910>): True}}

In [15]:
vdpinterpretation = {**labelconst_interpretation, **rel_interpretation}


In [16]:
model = FOModel(vdpvocabulary, vdpelements, vdpinterpretation)


In [17]:
tst = FOElement('x', vdpobject)
model.interpret(term=[vdprelations['labelOf'], [tst], [vdpconstants['cat']]], interpretation_extension={tst: vdpobjects['cat']})

True